<a href="https://colab.research.google.com/github/SuzumitsuTs1103/Moneyballing-Bank-Marketing/blob/master/IBM_Com_Wav2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# 日本語データセットのダウンロード
!pip install datasets==1.13.3
# 機械学習ライブラリ
!pip install transformers==4.11.3
!pip install torchaudio
# 音声データ処理用ライブラリ
!pip install librosa
# 形態素解析
!pip install mecab-python3
!pip install unidic
# かなローマ字変換
!pip install romkan


In [2]:
import torch


In [3]:
!python -m unidic download


download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:17<00:00, 29.7MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.7/dist-packages/unidic/dicdir


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jul 12 14:35:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice", "ja", split="train+validation")
common_voice_test = load_dataset("common_voice", "ja", split="test")


Downloading:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/153M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9. Subsequent calls will reuse this data.


Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/ja/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)


In [6]:
len(common_voice_train), len(common_voice_test)


(1308, 632)

In [7]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])


In [8]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)


In [9]:
import re
chars_to_ignore_regex = '[、,。]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)


Parameter 'function'=<function remove_special_characters at 0x7f26940e3290> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1308 [00:00<?, ?ex/s]

  0%|          | 0/632 [00:00<?, ?ex/s]

In [10]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(common_voice_test.remove_columns(["path","audio"]))


,sentence
0,このカレーはとても辛いです
1,全米パブリッシャーズ協会のベストストラテジーゲームオブザイヤーを日本人として受賞
2,この部屋に入ってはだめです
3,切符を買うのは自動販売機からである
4,立ってください
5,毎晩寝る前に本を読みます
6,そうなんです
7,辞書はあそこの本棚にあります
8,だれかわたしのボールペンを持っていませんか
9,屋根には気密式の電動ドリルで穴を開け生じる粉塵はどんどん吸引する


In [11]:
import MeCab
import unidic
import romkan
mecab = MeCab.Tagger()

sentence = common_voice_train[0]['sentence']
print(sentence)
print(mecab.parse(sentence))


予想外の事態に電力会社もちょっぴり困惑気味だ 
予想	名詞,普通名詞,サ変可能,,,,ヨソウ,予想,予想,ヨソー,予想,ヨソー,漢,"","","","","","",体,ヨソウ,ヨソウ,ヨソウ,ヨソウ,"0","C2","",10819203040944640,39360
外	接尾辞,名詞的,一般,,,,ガイ,外,外,ガイ,外,ガイ,漢,"","","","","","",接尾体,ガイ,ガイ,ガイ,ガイ,"","C3","",2169894821044736,7894
の	助詞,格助詞,,,,,ノ,の,の,ノ,の,ノ,和,"","","","","","",格助,ノ,ノ,ノ,ノ,"","名詞%F1","",7968444268028416,28989
事態	名詞,普通名詞,一般,,,,ジタイ,事態,事態,ジタイ,事態,ジタイ,漢,"","","","","","",体,ジタイ,ジタイ,ジタイ,ジタイ,"1","C1","",4922247303275008,17907
に	助詞,格助詞,,,,,ニ,に,に,ニ,に,ニ,和,"","","","","","",格助,ニ,ニ,ニ,ニ,"","名詞%F1","",7745518285496832,28178
電力	名詞,普通名詞,一般,,,,デンリョク,電力,電力,デンリョク,電力,デンリョク,漢,"","","","","","",体,デンリョク,デンリョク,デンリョク,デンリョク,"0,1","C2","",7095706913481216,25814
会社	名詞,普通名詞,一般,,,,カイシャ,会社,会社,カイシャ,会社,カイシャ,漢,"カ濁","基本形","","","","",体,カイシャ,カイシャ,カイシャ,カイシャ,"0","C2","",1577258053673472,5738
も	助詞,係助詞,,,,,モ,も,も,モ,も,モ,和,"","","","","","",係助,モ,モ,モ,モ,"","動詞%F2@-1,形容詞%F4@-2,名詞%F1","",10324972564259328,37562
ちょっぴり	副詞,,,,,,チョッピリ,ちょっぴり,ちょっぴり,チョッピリ,ちょっぴり,チョッピリ,和,"","","","","","",相,チョッピリ,チョッピリ,チョッピリ,チョッピリ,"3","",

In [12]:
def convert_sentence_to_kana(batch):
    s = mecab.parse(batch["sentence"])
    kana = ""
    for line in s.split("\n"):
      if line.find("\t")<=0: continue
      columns = line.split(',')
      if len(columns) < 10:
        kana += line.split('\t')[0]
      else:
        kana += columns[9]
    batch["kana"] = kana
    return batch


In [13]:
common_voice_train = common_voice_train.map(convert_sentence_to_kana)
common_voice_test = common_voice_test.map(convert_sentence_to_kana)

  0%|          | 0/1308 [00:00<?, ?ex/s]

  0%|          | 0/632 [00:00<?, ?ex/s]

In [14]:
show_random_elements(common_voice_test.remove_columns(["path","audio"]))


,sentence,kana
0,松井さんは田中さんの誕生日にネクタイをあげます,マツイサンワタナカサンノタンジョーヒニネクタイオアゲマス
1,なぜ日本語を勉強しているんですか,ナゼニッポンゴオベンキョーシテイルンデスカ
2,私はいろいろの弁護を自分の胸でこしらえてみました,ワタクシワイロイロノベンゴオジブンノムネデコシラエテミマシタ
3,山田さんはおもしろい人です,ヤマダサンワオモシロイヒトデス
4,屋根には気密式の電動ドリルで穴を開け生じる粉塵はどんどん吸引する,ヤネニワキミツシキノデンドードリルデアナオアケショージルフンジンワドンドンキューインスル
5,限られた予算で仕事をする美術スタッフの気持ちをおしはかって我慢する,カギラレタヨサンデシゴトオスルビジュツスタッフノキモチオオシハカッテガマンスル
6,あの店は安いですがまずいです,アノミセワヤスイデスガマズイデス
7,いずれはハワイやカリフォルニアなど日本人が多く住む暑い土地で育ててみたい,イズレワハワイヤカリフォルニアナドニッポンジンガオークスムアツイトチデソダテテミタイ
8,もっと広い議場をという声もあったがチャーチル首相が抑えた,モットヒロイギジョーオトユーコエモアッタガチャーチルシュショーガオサエタ
9,あしたは午前中学校にいます,アシタワゴゼンチューガッコーニイマス


In [15]:
def convert_sentence_to_roman(batch):
    s = mecab.parse(batch["kana"])
    kana = ""
    for line in s.split("\n"):
      if line.find("\t")<=0: continue
      columns = line.split(',')
      if len(columns) < 10:
        kana += line.split('\t')[0]
      else:
        kana += columns[9]
    roman = romkan.to_roma(kana)
    batch["roman"] = roman
    return batch


In [16]:
common_voice_train = common_voice_train.map(convert_sentence_to_roman)
common_voice_test = common_voice_test.map(convert_sentence_to_roman)

  0%|          | 0/1308 [00:00<?, ?ex/s]

  0%|          | 0/632 [00:00<?, ?ex/s]

In [17]:
# "kana" は邪魔なので除いておく
show_random_elements(common_voice_test.remove_columns(["path","audio", "kana"])) 


,sentence,roman
0,きょうは勉強しなければなりませんがあしたは大丈夫です,kyowabenkyo-shinakerebanarimasengaashitawadaijo-budesu
1,あしたテストがあるので今晩は勉強します,ashitatesutogaarunodekonbanwabenkyo-shimasu
2,このコンピューターは高いですがとてもいいです,konokonpyu-ta-watakaidesugatotemoi-desu
3,ガラス扉を押して入ると一瞬首や肘から汗の膜が剥ぎとられる,garasutobirao-shitehairutoisshunkubiyahijikaraasenomakugahagitorareru
4,妹はきょうも高い熱が続いているので心配です,imo-towakyo-motakainetsugatsuzuiteirunodeshinpaidesu
5,生きてっか？,ikitekka
6,しかしその応援ぶりはヤケ気味にかえって激しくなった,shikashisonoo-enburiwayakegiminikaettehageshikunatta
7,けさは時間があるので新聞を読んでから家を出ます,kesawajikangaarunodeshinbun'oyondekaraieodemasu
8,事実とすればよほど頭の回転の速い人だろう,jijitsutosurebayohodoatamanokaiten'nohayaihitodaro-
9,不思議なくらい美しく彩られた道を行く,fushiginakuraiutsukushikuirodoraretamichioiku


In [18]:
def extract_all_chars_sentence(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

def extract_all_chars_kana(batch):
  all_text = " ".join(batch["kana"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

def extract_all_chars_roman(batch):
  all_text = " ".join(batch["roman"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [19]:
vocab_train_sentence = common_voice_train.map(extract_all_chars_sentence, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test_sentence = common_voice_test.map(extract_all_chars_sentence, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)
vocab_list_sentence = list(set(vocab_train_sentence["vocab"][0]) | set(vocab_test_sentence["vocab"][0]))
vocab_dict_sentence = {v: k for k, v in enumerate(vocab_list_sentence)}

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:

vocab_train_kana = common_voice_train.map(extract_all_chars_kana, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test_kana = common_voice_test.map(extract_all_chars_kana, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)
vocab_list_kana = list(set(vocab_train_kana["vocab"][0]) | set(vocab_test_kana["vocab"][0]))
vocab_dict_kana = {v: k for k, v in enumerate(vocab_list_kana)}

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [21]:
vocab_train_roman = common_voice_train.map(extract_all_chars_roman, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test_roman = common_voice_test.map(extract_all_chars_roman, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)
vocab_list_roman = list(set(vocab_train_roman["vocab"][0]) | set(vocab_test_roman["vocab"][0]))
vocab_dict_roman = {v: k for k, v in enumerate(vocab_list_roman)}


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [22]:
len(vocab_dict_sentence), len(vocab_dict_kana), len(vocab_dict_roman)


(1432, 82, 30)

In [23]:
vocab_dict = vocab_dict_roman.copy()

In [24]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)


In [25]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


In [26]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)


In [27]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [28]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))


In [29]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]

    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [30]:

common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, num_proc=4)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, num_proc=4)


In [31]:
import torch

# from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

In [32]:
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")


In [33]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [34]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Config

configuration = Wav2Vec2Config(hidden_size=256, num_hidden_layers=6, num_attention_heads=6, intermediate_size=1024)

model_args ={}


In [35]:
# from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)


Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_hid.bias', 'quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to u

In [36]:
model.config = configuration


In [37]:
# Fine Tuning で Feature Extractor が変化しないように設定を入れる
model.freeze_feature_extractor()

In [38]:
model = model.to(torch.device("cpu"))


In [39]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./wav2vec2-large-xlsr-japanese-demo",
  group_by_length=True,
  per_device_train_batch_size=8,   #16
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  # num_train_epochs=30,
  num_train_epochs=3,
  # fp16=True,
  save_steps=10,    #100
  eval_steps=10,     #100
  logging_steps=2,  #10
  learning_rate=3e-4,
  warmup_steps=50,    #500
  save_total_limit=2,
)


In [40]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)


In [41]:
torch.cuda.empty_cache()


In [42]:
train_result = trainer.train()

***** Running training *****
  Num examples = 1308
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 246
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  tensor = as_tensor(value)


RuntimeError: ignored

In [ ]:
metrics = train_result.metrics
max_train_samples = len(_common_voice_train)
metrics["train_samples"] = min(max_train_samples, len(_common_voice_train))

trainer.save_model()



# https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLSR_Wav2Vec2_on_Common_Voice.ipynb

In [1]:
references = [
    {"path": "/path/to/sagan.mp3", "transcription": "extraordinary claims require extraordinary evidence"},
    {"path": "/path/to/asimov.wav", "transcription": "violence is the last refuge of the incompetent"},
]


In [3]:
!pip install  pyopenjtalk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyopenjtalk: filename=pyopenjtalk-0.2.0-cp37-cp37m-linux_x86_64.whl size=4428163 sha256=991091657b86d603ad325059834abffe21c538aae240f8551e4081205cf413da
  Stored in directory: /root/.cache/pip/wheels/10/56/0e/435dc1aec0d8614a489abfc51da4fd54ff6e8b33bf978f2081
Successfully built pyopenjtalk


In [5]:
import json
import random
import librosa
import pyopenjtalk
import pandas as pd
import soundfile as sf

# グローバル設定
class GCF:
    # git cloneで取得した声庭ファイル
    KONIWA_ROOT = './koniwa/data'
    # アノテーション済みのデータリスト
    ANNOTATION_DATA = [
        'amagasaki/amagasaki__2011_04_20',
        'free_culture_2012/free_culture_2012__1',
        'tnc/tnc__gongitsune',
        'tnc/tnc__kumonoito',
        'tnc/tnc__nezuminoyomeiri',
        'tnc/tnc__tebukurowokaini',
    ]
    # モデルの出力先
    OUTPUT_DIR = f"./output/finetuned"
    # 前処理済みデータの保存先
    PREPROCESSED = f"./output/preprocessed"
    # 評価データ数
    N_EVAL = 200

audio_data = []
for annotation in GCF.ANNOTATION_DATA:
    d = json.load(open(f'{GCF.KONIWA_ROOT}/{annotation}.json', 'r'))
    df = pd.DataFrame(d['annotation'])
    # pyopenjtalkを使って音素変換
    df['phoneme'] = df['data'].map(lambda x: pyopenjtalk.g2p(x['kana_level0'], kana=False).lower())
    # tncの音声データのみ拡張子が.aacでそれ以外は.mp3
    suffix = 'aac' if 'tnc' == annotation[:3] else 'mp3'
    a, sr = librosa.load(f"./{annotation}.{suffix}")
    # フレーズごとに音声ファイルを分割する
    for idx, row in df.iterrows():
        i = int(row.start * sr)
        j = int(row.end * sr)
        speech = a[i:j]
        fname = annotation.split('/')[-1]
        p = f'{GCF.PREPROCESSED}/{fname}_{idx:04}.wav'
        sf.write(p, speech, sr, 'PCM_24')
        r = {"path": p, "transcription": row['phoneme']}
        audio_data.append(r)

df = pd.DataFrame(audio_data).dropna()
audio_data = [row.to_dict() for _, row in df.iterrows()]
# 短すぎる音声は除外する
audio_data = [i for i in audio_data if len(i['transcription']) > 10]

# 学習データと評価データに分割
random.shuffle(audio_data)
eval_data, train_data= audio_data[:GCF.N_EVAL], audio_data[GCF.N_EVAL:]


FileNotFoundError: ignored

In [ ]:
model = SpeechRecognitionModel("facebook/wav2vec2-large-xlsr-53", device=device)
tokens = list('abcdefghijklmnopqrstuvwxyz ')
token_set = TokenSet(tokens)

training_args = TrainingArguments(
    learning_rate=3e-4,
    max_steps=1000,
    eval_steps=200,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
)
model_args = ModelArguments(
    activation_dropout=0.1,
    hidden_dropout=0.1,
)

model.finetune(
    GCF.OUTPUT_DIR, 
    train_data=train_data, 
    eval_data=eval_data,
    token_set=token_set, 
    training_args=training_args,
    model_args=model_args,
)


In [ ]:
evaluation = model.evaluate(eval_data, inference_batch_size=1)
print(evaluation)
# {'cer': 0.22771011877913097, 'wer': 0.282102666872206}


In [ ]:
audio_paths = ['test_audio.mp3']
transcriptions = model.transcribe(audio_paths, batch_size=1)
